In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
comment = pd.read_csv('./train.csv')
comment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
id               159571 non-null object
comment_text     159571 non-null object
toxic            159571 non-null int64
severe_toxic     159571 non-null int64
obscene          159571 non-null int64
threat           159571 non-null int64
insult           159571 non-null int64
identity_hate    159571 non-null int64
dtypes: int64(6), object(2)
memory usage: 8.5+ MB


In [ ]:
comment.head()

In [8]:
print comment.toxic.sum()
print comment.severe_toxic.sum()
print comment.obscene.sum()
print comment.threat.sum()
print comment.insult.sum()
print comment.identity_hate.sum()

15294
1595
8449
478
7877
1405


In [ ]:
#extracting the stopwords from nltk library
sw = stopwords.words('english')
# displaying the stopwords
np.array(sw)

In [ ]:
def stopwords(text):
    '''a function for removing the stopword'''
# removing the stop words and lowercasing the selected words
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    # joining the list of words with space separator
    return " ".join(text)

In [ ]:
all_text['comment_text'] = all_text['comment_text'].apply(stopwords)
all_text.head(10)

In [85]:
# create an object of stemming function
stemmer = SnowballStemmer("english")
def stemming(text):
    '''a function which stems each word in the given text'''
    text = [stemmer.stem(word) for word in text.split()]
    return " ".join(text)

In [93]:
comment.comment_text = map(lambda x:unicode(x, 'utf-8'), comment.comment_text)

In [94]:
#Dont run. not working in this case
#comment['comment_text'] = unicode(comment['comment_text'], 'utf-8')
comment['comment_text'] = comment['comment_text'].apply(stemming)
comment.head(10)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explan whi the edit made under my usernam hard...,0,0,0,0,0,0
1,000103f0d9cfb60f,d'aww! he match this background colour i'm see...,0,0,0,0,0,0
2,000113f07ec002fd,"hey man, i'm realli not tri to edit war. it ju...",0,0,0,0,0,0
3,0001b41b1c6bb37e,""" more i can't make ani real suggest on improv...",0,0,0,0,0,0
4,0001d958c54c6e35,"you, sir, are my hero. ani chanc you rememb wh...",0,0,0,0,0,0
5,00025465d4725e87,""" congratul from me as well, use the tool well...",0,0,0,0,0,0
6,0002bcb3da6cb337,cocksuck befor you piss around on my work,1,1,1,0,1,0
7,00031b1e95af7921,your vandal to the matt shirvington articl has...,0,0,0,0,0,0
8,00037261f536c51d,sorri if the word nonsens was offens to you. a...,0,0,0,0,0,0
9,00040093b2687caa,align on this subject and which are contrari t...,0,0,0,0,0,0


In [3]:
col = comment.columns.tolist()
col
X = comment[col[0:2]]
y = comment[col[2:8]]
print X.shape
print y.shape
#print X
#print y

(159571, 2)
(159571, 6)


In [3]:
# Import Pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
# Import other necessary modules
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

In [4]:
from warnings import warn

import numpy as np
import pandas as pd

def multilabel_sample(y, size=1000, min_count=5, seed=None):
    """ Takes a matrix of binary labels `y` and returns
        the indices for a sample of size `size` if
        `size` > 1 or `size` * len(y) if size =< 1.
        The sample is guaranteed to have > `min_count` of
        each label.
    """
    try:
        if (np.unique(y).astype(int) != np.array([0, 1])).all():
            raise ValueError()
    except (TypeError, ValueError):
        raise ValueError('multilabel_sample only works with binary indicator matrices')

    if (y.sum(axis=0) < min_count).any():
        raise ValueError('Some classes do not have enough examples. Change min_count if necessary.')

    if size <= 1:
        size = np.floor(y.shape[0] * size)

    if y.shape[1] * min_count > size:
        msg = "Size less than number of columns * min_count, returning {} items instead of {}."
        warn(msg.format(y.shape[1] * min_count, size))
        size = y.shape[1] * min_count

    rng = np.random.RandomState(seed if seed is not None else np.random.randint(1))

    if isinstance(y, pd.DataFrame):
        choices = y.index
        y = y.values
    else:
        choices = np.arange(y.shape[0])

    sample_idxs = np.array([], dtype=choices.dtype)

    # first, guarantee > min_count of each label
    for j in range(y.shape[1]):
        label_choices = choices[y[:, j] == 1]
        label_idxs_sampled = rng.choice(label_choices, size=min_count, replace=False)
        sample_idxs = np.concatenate([label_idxs_sampled, sample_idxs])

    sample_idxs = np.unique(sample_idxs)

    # now that we have at least min_count of each, we can just random sample
    sample_count = int(size - sample_idxs.shape[0])

    # get sample_count indices from remaining choices
    remaining_choices = np.setdiff1d(choices, sample_idxs)
    remaining_sampled = rng.choice(remaining_choices,
                                   size=sample_count,
                                   replace=False)

    return np.concatenate([sample_idxs, remaining_sampled])


def multilabel_sample_dataframe(df, labels, size, min_count=5, seed=None):
    """ Takes a dataframe `df` and returns a sample of size `size` where all
        classes in the binary matrix `labels` are represented at
        least `min_count` times.
    """
    idxs = multilabel_sample(labels, size=size, min_count=min_count, seed=seed)
    return df.loc[idxs]


def multilabel_train_test_split(X, Y, size, min_count=5, seed=None):
    """ Takes a features matrix `X` and a label matrix `Y` and
        returns (X_train, X_test, Y_train, Y_test) where all
        classes in Y are represented at least `min_count` times.
    """
    index = Y.index if isinstance(Y, pd.DataFrame) else np.arange(Y.shape[0])

    test_set_idxs = multilabel_sample(Y, size=size, min_count=min_count, seed=seed)
    train_set_idxs = np.setdiff1d(index, test_set_idxs)

    test_set_mask = index.isin(test_set_idxs)
    train_set_mask = ~test_set_mask

    return (X[train_set_mask], X[test_set_mask], Y[train_set_mask], Y[test_set_mask])


In [20]:
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

# Split out only the text data
X_train, X_valid, y_train, y_valid = multilabel_train_test_split(X['comment_text'], y, size=0.3, seed=123)

# Instantiate Pipeline object: pl
pl = Pipeline([
        
        ('vec', CountVectorizer(stop_words='english')),
        
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_valid, y_valid)
print("\nAccuracy on sample data - just text data: ", accuracy)


('\nAccuracy on sample data - just text data: ', 0.90796097846295254)


In [8]:
print X_train.shape
print X_valid.shape
print y_train.shape
print y_valid.shape

(111700,)
(47871,)
(111700, 6)
(47871, 6)


In [13]:
print X_valid.head()
pl.predict_proba(X_valid)

2     Hey man, I'm really not trying to edit war. It...
3     "\nMore\nI can't make any real suggestions on ...
9     alignment on this subject and which are contra...
10    "\nFair use rationale for Image:Wonju.jpg\n\nT...
14    Oh, and the girl above started her arguments w...
Name: comment_text, dtype: object


array([[ 0.04352204,  0.00096882,  0.03068547,  0.00032574,  0.02698306,
         0.0013083 ],
       [ 0.00020962,  0.00033909,  0.00137987,  0.00015485,  0.00149037,
         0.00128533],
       [ 0.06736556,  0.01483952,  0.04718971,  0.00492352,  0.0726206 ,
         0.01405798],
       ..., 
       [ 0.01185786,  0.0030648 ,  0.0147588 ,  0.00179327,  0.01644665,
         0.00543816],
       [ 0.00016032,  0.0007825 ,  0.00106099,  0.00049674,  0.00091813,
         0.00129503],
       [ 0.08584951,  0.00455111,  0.03502991,  0.00241613,  0.03122413,
         0.00557291]])

In [21]:
# Split out only the text data
X_train, X_valid, y_train, y_valid = multilabel_train_test_split(X['comment_text'], y, size=0.3, seed=123)

# Instantiate Pipeline object: pl
pl = Pipeline([
        ('vec', CountVectorizer(stop_words='english')),
        ('clf', OneVsRestClassifier(RandomForestClassifier(n_estimators=15)))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_valid, y_valid)
print("\nAccuracy on sample data - just text data: ", accuracy)


('\nAccuracy on sample data - just text data: ', 0.91017526268513294)


In [8]:
from sklearn.naive_bayes import MultinomialNB
# Split out only the text data
X_train, X_valid, y_train, y_valid = multilabel_train_test_split(X['comment_text'], y, size=0.3, seed=123)

# Instantiate Pipeline object: pl
pl = Pipeline([
        #('scaler', StandardScaler()),
        ('vec', CountVectorizer(stop_words='english')),
        ('clf', OneVsRestClassifier(MultinomialNB()))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_valid, y_valid)
print("\nAccuracy on sample data - just text data: ", accuracy)


('\nAccuracy on sample data - just text data: ', 0.90570491529318375)


In [13]:
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

# Split out only the text data
X_train, X_valid, y_train, y_valid = multilabel_train_test_split(X['comment_text'], y, size=0.3, seed=123)

# Instantiate Pipeline object: pl
pl = Pipeline([
        
        ('vec', TfidfVectorizer(stop_words='english')),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_valid, y_valid)
print("\nAccuracy on sample data - just text data: ", accuracy)


('\nAccuracy on sample data - just text data: ', 0.91725679430135154)


In [6]:
TOKENS_ALPHANUMERIC = '[A-Za-z0-9]+(?=\\s+)'

# Split out only the text data
X_train, X_valid, y_train, y_valid = multilabel_train_test_split(X['comment_text'], y, size=0.3, seed=123)

# Instantiate Pipeline object: pl
pl = Pipeline([
        
        ('vec', TfidfVectorizer(stop_words='english')),
        ('clf', OneVsRestClassifier(SVC()))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_valid, y_valid)
print("\nAccuracy on sample data - just text data: ", accuracy)


('\nAccuracy on sample data - just text data: ', 0.8965344362975497)


In [24]:
X_train, X_valid, y_train, y_valid = multilabel_train_test_split(X['comment_text'], y, size=0.3, seed=123)

# Instantiate Pipeline object: pl
pl = Pipeline([
        #('scaler', StandardScaler()),
        ('vec', TfidfVectorizer(min_df=0.00009,  strip_accents='unicode', encoding='utf-8', analyzer='word',token_pattern = r'\w{1,}', ngram_range=(1,1), sublinear_tf=True, stop_words='english')),
        ('clf', OneVsRestClassifier(LogisticRegression(solver='sag')))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_valid, y_valid)
print("\n Accuracy on sample data - just text data: ", accuracy)

('\nAccuracy on sample data - just text data: ', 0.9181550416745002)


In [22]:
X_train, X_valid, y_train, y_valid = train_test_split(X['comment_text'], y, test_size=0.3, random_state=42)

# Instantiate Pipeline object: pl
pl = Pipeline([
        #('scaler', StandardScaler()),
        ('vec', TfidfVectorizer(min_df=0.00009,  strip_accents='unicode', encoding='utf-8', analyzer='word',token_pattern = r'\w{1,}', ngram_range=(1,1), sublinear_tf=1, stop_words='english')),
        ('clf', OneVsRestClassifier(LogisticRegression(solver='sag')))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_valid, y_valid)
print("\nAccuracy on sample data - just text data: ", accuracy)

('\nAccuracy on sample data - just text data: ', 0.9184909759358288)


In [7]:
# Split out only the text data
X_train, X_valid, y_train, y_valid = multilabel_train_test_split(X['comment_text'], y, size=0.3, seed=123)

# Instantiate Pipeline object: pl
pl = Pipeline([
        
        ('vec', TfidfVectorizer(stop_words='english')),
        ('clf', OneVsRestClassifier(GradientBoostingClassifier()))
    ])

# Fit to the training data
pl.fit(X_train, y_train)

# Compute and print accuracy
accuracy = pl.score(X_valid, y_valid)
print("\nAccuracy on sample data - just text data: ", accuracy)


('\nAccuracy on sample data - just text data: ', 0.90829521004365898)
